# Business Case 3 - Recommender System

- Emanuele Aldera, m20210617
- Robin Schmidt, m20210602
- Muhammad Abdullah, m20210655
- Rui Ramos, m20210640

## Importing Libraries and Dataset

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil 
from datetime import datetime
from mlxtend.frequent_patterns import apriori, association_rules
from tabulate import tabulate

In [51]:
data = pd.read_csv("retail.csv")
data_orig = data.copy()
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [52]:
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])
data["Year"] = data["InvoiceDate"].dt.year
data["Month"] = data["InvoiceDate"].dt.month
# data["MonthName"] = data["InvoiceDate"].dt.month_name()
data["Day"] = data["InvoiceDate"].dt.day
data["Weekday"] = data["InvoiceDate"].dt.weekday +1
data["WeekdayName"] = data["InvoiceDate"].dt.day_name()
data["IsWeekend"] = data["Weekday"] > 5
data["Time"] = data["InvoiceDate"].dt.time
data["Hour"] = data["InvoiceDate"].dt.hour
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   Year         541909 non-null  int64         
 9   Month        541909 non-null  int64         
 10  Day          541909 non-null  int64         
 11  Weekday      541909 non-null  int64         
 12  WeekdayName  541909 non-null  object        
 13  IsWeekend    541909 non-null  bool          
 14  Time         541909 non-null  object        
 15  Hour         541909 non-null  int6

In [53]:
def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

prods = pd.DataFrame({'hour':range(1, 25)})

b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
prods['session'] = pd.cut(prods['hour'], bins=b, labels=l, include_lowest=True)

data["DayTime"] = data['Hour'].apply(f)

In [54]:
prods = pd.DataFrame({'hour':range(1, 25)})

b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
prods['session'] = pd.cut(prods['hour'], bins=b, labels=l, include_lowest=True)

In [55]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0
Year,541909.0,2010.921609,0.268787,2010.00,2011.00,2011.00,2011.00,2011.0
Month,541909.0,7.553128,3.509055,1.00,5.00,8.00,11.00,12.0
Day,541909.0,15.023096,8.664063,1.00,7.00,15.00,22.00,31.0
Weekday,541909.0,3.431277,1.844709,1.00,2.00,3.00,5.00,7.0
Hour,541909.0,13.078729,2.443270,6.00,11.00,13.00,15.00,20.0


## Data exploration

Cancelations

In [56]:
cancelations = data.loc[data["InvoiceNo"].str.contains("C")]
cancelations

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,2010,12,1,3,Wednesday,False,09:41:00,9,Morning
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,2010,12,1,3,Wednesday,False,09:49:00,9,Morning
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,2010,12,1,3,Wednesday,False,10:24:00,10,Morning
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,2010,12,1,3,Wednesday,False,10:24:00,10,Morning
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,2010,12,1,3,Wednesday,False,10:24:00,10,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,2011,12,9,5,Friday,False,09:57:00,9,Morning
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,2011,12,9,5,Friday,False,10:28:00,10,Morning
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,2011,12,9,5,Friday,False,11:57:00,11,Morning
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,2011,12,9,5,Friday,False,11:58:00,11,Morning


In [57]:
#check consistency for cancelations, we shouldnt have any cancelations with quantity > 0
cancelations.loc[cancelations["Quantity"] > 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime


In [58]:
#check if for cancellations we have also the non cancelled record
cancelations2 = cancelations.copy()

#deleting C into cancelled invoices to have a joinable column to original df
cancelations2["InvoiceNo"] = cancelations2["InvoiceNo"].str.replace("C", "")

#results finds 0 records --> no duplicated records
pd.merge(left = data, right = cancelations2, how = "inner", on = "InvoiceNo")

,InvoiceNo,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,Year_x,Month_x,...,Country_y,Year_y,Month_y,Day_y,Weekday_y,WeekdayName_y,IsWeekend_y,Time_y,Hour_y,DayTime_y


In [59]:
#removing from dataset cancelations
data = data[data["Quantity"] >= 0]

Missing values

In [60]:
data.isna().sum(axis = 0)

InvoiceNo           0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133361
Country             0
Year                0
Month               0
Day                 0
Weekday             0
WeekdayName         0
IsWeekend           0
Time                0
Hour                0
DayTime             0
dtype: int64

In [61]:
#showing records not having customer id
data[data.CustomerID.isna()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom,2010,12,1,3,Wednesday,False,11:52:00,11,Morning
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:32:00,14,Noon
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:32:00,14,Noon
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:32:00,14,Noon
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:32:00,14,Noon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom,2011,12,9,5,Friday,False,10:26:00,10,Morning
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom,2011,12,9,5,Friday,False,10:26:00,10,Morning
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom,2011,12,9,5,Friday,False,10:26:00,10,Morning
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom,2011,12,9,5,Friday,False,10:26:00,10,Morning


In [62]:
# number of invoices not having at least one missing customer id value
len(data[data.CustomerID.isna()]["InvoiceNo"].unique())

2192

Data Consistency

In [63]:
#checking dataset consistency - for each invoice only one customer id associated
grouped = data.groupby("InvoiceNo")["CustomerID"].nunique(dropna=False)
grouped.sort_values(ascending = False)

InvoiceNo
536365     1
566872     1
566822     1
566821     1
566819     1
          ..
551946     1
551945     1
551944     1
551907     1
A563187    1
Name: CustomerID, Length: 20728, dtype: int64

In [64]:
#checking dataset consistency - for each stock code only one description associated
grouped = data.groupby("StockCode")["Description"].nunique(dropna=False)
grouped.sort_values(ascending = False)

StockCode
23084    7
20713    7
21181    5
22734    5
84944    4
        ..
22528    1
22529    1
22530    1
22531    1
m        1
Name: Description, Length: 3941, dtype: int64

In [65]:
# by looking at results we decide not to take care of this issue, we just use stock code
data[data["StockCode"] == "20713"]["Description"].unique()

array(['JUMBO BAG OWLS', nan, 'found', 'Found', 'wrongly marked 23343',
       'Marked as 23343', 'wrongly coded 23343'], dtype=object)

In [66]:
data[data.UnitPrice == 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom,2010,12,1,3,Wednesday,False,11:52:00,11,Morning
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:32:00,14,Noon
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:33:00,14,Noon
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:33:00,14,Noon
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom,2010,12,1,3,Wednesday,False,14:34:00,14,Noon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535334,581211,22142,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom,2011,12,7,3,Wednesday,False,18:36:00,18,Eve
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom,2011,12,8,4,Thursday,False,10:33:00,10,Morning
538504,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom,2011,12,8,4,Thursday,False,13:58:00,13,Noon
538505,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom,2011,12,8,4,Thursday,False,13:58:00,13,Noon


In [67]:
#number of products having unit price to 0
print("# of rows having price 0 : ", len(data[data.UnitPrice == 0]))
print("# of products having price 0 : ", len(data[data.UnitPrice == 0]["StockCode"].unique()))

# of rows having price 0 :  1179
# of products having price 0 :  681


In [68]:
#Reducing cardinality for Country
data.loc[data["Country"].value_counts()[data["Country"]].values < 500, "Country"] = "Others"

Duplicates

In [69]:
data_dup = data.copy()
print("Percentage of suspect duplicated data: " , (data_orig.duplicated(["StockCode", "InvoiceNo"]).sum() / len(data)) * 100)

Percentage of suspect duplicated data:  2.0109733946939965


In [21]:
duplicates = data_dup.loc[data_dup.duplicated()]
duplicates = duplicates.groupby(duplicates.columns.tolist(),as_index=False).size().sort_values("size", ascending = False)
size = duplicates.pop("size")
duplicates.insert(0, "size", size)
duplicates

,size,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
1596,19,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
1595,11,555524,22697,GREEN REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
3171,7,572861,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-10-26 12:46:00,1.25,14102.0,United Kingdom,2011,10,26,3,Wednesday,False,12:46:00,12,Morning
525,5,541266,21755,LOVE BUILDING BLOCK WORD,1,2011-01-16 16:25:00,5.95,15673.0,United Kingdom,2011,1,16,7,Sunday,True,16:25:00,16,Noon
346,5,538514,21756,BATH BUILDING BLOCK WORD,1,2010-12-12 14:27:00,5.95,15044.0,United Kingdom,2010,12,12,7,Sunday,True,14:27:00,14,Noon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,1,556280,22500,SET OF 2 TINS JARDIN DE PROVENCE,1,2011-06-09 20:03:00,4.95,14584.0,United Kingdom,2011,6,9,4,Thursday,False,20:03:00,20,Eve
1658,1,556280,22497,SET OF 2 TINS VINTAGE BATHROOM,1,2011-06-09 20:03:00,4.25,14584.0,United Kingdom,2011,6,9,4,Thursday,False,20:03:00,20,Eve
1657,1,556263,23242,TREASURE TIN BUFFALO BILL,1,2011-06-09 18:41:00,2.08,13969.0,United Kingdom,2011,6,9,4,Thursday,False,18:41:00,18,Eve
1656,1,556252,84970S,HANGING HEART ZINC T-LIGHT HOLDER,12,2011-06-09 16:44:00,0.85,16779.0,United Kingdom,2011,6,9,4,Thursday,False,16:44:00,16,Noon


In [22]:
data[(data.InvoiceNo == "555524") & (data.StockCode == "22698")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
213176,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213177,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213183,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213191,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213192,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213194,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213195,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213196,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213197,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning
213198,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning


In [23]:
#replacing duplicates with single record containing sum of all quantities for records that are going to be deleted
# tested, it works
data['Quantity'] = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].transform('sum')
data = data.drop_duplicates(subset=['InvoiceNo', 'StockCode'])

C:\Users\emanu\AppData\Local\Temp\ipykernel_15024\937976445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Quantity'] = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].transform('sum')


In [24]:
data[(data.InvoiceNo == "555524") & (data.StockCode == "22698")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Weekday,WeekdayName,IsWeekend,Time,Hour,DayTime
213176,555524,22698,PINK REGENCY TEACUP AND SAUCER,20,2011-06-05 11:37:00,2.95,16923.0,United Kingdom,2011,6,5,7,Sunday,True,11:37:00,11,Morning


In [25]:
data["Sales"] = data["UnitPrice"] * data["Quantity"]

ABC segmentation

In [26]:
# leverage = data[data["Quantity"] >= 0].groupby("CustomerID").sum()
# leverage["TotalValueCustomer"] = leverage["Quantity"] * leverage["UnitPrice"]
# leverage.sort_values("TotalValueCustomer", inplace = True, ascending=False)
# leverage.reset_index(drop=False, inplace = True)
# total_profit = leverage["TotalValueCustomer"].sum(axis = 0)
# leverage["PercentProfit"] = (leverage["TotalValueCustomer"] / total_profit)
# leverage

In [27]:
# data[(data.CustomerID == 14911.0) & (data.Quantity * data.UnitPrice > 100)]

In [28]:
#Check Toghether results
# sns.lineplot(x = leverage.index, y = leverage.TotalValueCustomer)

In [29]:
# sns.boxplot(leverage["TotalValueCustomer"])

In [30]:
# sns.lineplot(x = leverage[leverage.TotalValueCustomer < 1000000].index, y = leverage[leverage.TotalValueCustomer < 1000000].PercentProfit)

## A priori algorithm

In [44]:
basket = (data.groupby(['InvoiceNo', 'StockCode'])['Quantity']
        .sum().unstack().reset_index().fillna(0)
        .set_index('InvoiceNo'))
basket

# Transactions done in France
basket_France = (data[data['Country'] =="France"]
        .groupby(['InvoiceNo', 'StockCode'])['Quantity']
        .sum().unstack().reset_index().fillna(0)
        .set_index('InvoiceNo'))

In [45]:
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# basket = basket.applymap(hot_encode)
# basket
basket_France = basket_France.applymap(hot_encode)

In [46]:
# frq_items = apriori(basket, min_support = 0.03, use_colnames = True)
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)

In [47]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

rules.head(16)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
45,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000000,1.306667,0.017961,inf
260,"(PLASTERS IN TIN CIRCUS PARADE , RED TOADSTOOL...",(POSTAGE),0.051020,0.765306,0.051020,1.000000,1.306667,0.011974,inf
272,"(PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...",(POSTAGE),0.053571,0.765306,0.053571,1.000000,1.306667,0.012573,inf
302,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
300,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
334,"(POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS ,...",(SET/6 RED SPOTTY PAPER PLATES),0.084184,0.127551,0.081633,0.969697,7.602424,0.070895,28.790816
335,"(POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS ,...",(SET/6 RED SPOTTY PAPER CUPS),0.084184,0.137755,0.081633,0.969697,7.039282,0.070036,28.454082
129,(RED RETROSPOT PICNIC BAG),(POSTAGE),0.071429,0.765306,0.068878,0.964286,1.260000,0.014213,6.571429
143,(SET OF 9 BLACK SKULL BALLOONS),(POSTAGE),0.066327,0.765306,0.063776,0.961538,1.256410,0.013015,6.102041
174,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122


In [48]:
len(rules)

348